Implementation based on the following papers, articles and repositories:

- [**(Article)** Combining GPT-2 and BERT to make a fake person](https://bonkerfield.org/2020/02/combining-gpt-2-and-bert/)

- [**(Article)** How to build a convincing reddit personality with GPT2 and BERT](https://bonkerfield.org/2020/02/reddit-bot-gpt2-bert/)

- [**(Repository)** gpt2-bert-reddit-bot](https://github.com/lots-of-things/gpt2-bert-reddit-bot)

- [**(Paper)** Hierarchical Transformers for Long Document Classification](https://arxiv.org/abs/1910.10781)

- [**(Article)** Text Classification with BERT using Transformers for long text inputs](https://medium.com/analytics-vidhya/text-classification-with-bert-using-transformers-for-long-text-inputs-f54833994dfd)

- [**(Article)** Using BERT For Classifying Documents with Long Texts](https://medium.com/@armandj.olivares/using-bert-for-classifying-documents-with-long-texts-5c3e7b04573d)

- [**(Repository)** bert_for_long_text](https://github.com/ArmandDS/bert_for_long_text)

In [ ]:
%tensorflow_version 1.x
import os
import tarfile
import shutil

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
!pip install bert-tensorflow
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
# get input data
train = pd.read_csv('TODO.csv')
# train = train.reindex(np.random.permutation(train.index))
# train.head()

In [ ]:
# THIS CELL TAKES THE 'label' COLUMN AND MAKES IT NUMERIC (I.E. IN CASE ITS TEXT)

# # numerize label
# from sklearn.preprocessing import LabelEncoder
# LE = LabelEncoder()
# train_raw['label'] = LE.fit_transform(train_raw['label'])
# train_raw.head()

In [ ]:
# add label column to train data (which is legit, so always 1)
train['label'] = 1

In [ ]:
# remove non alphanumeric characters from the 'text' column
import re
def clean_txt(text):
    text = re.sub("'", "",text)
    text=re.sub("(\\W)+"," ",text)    
    return text

train['text']  = train.text.apply(clean_txt)
# train.head()

In [ ]:
fakes = []
for i in ["100", "200", "400", "600", "800"]:
    fakes.append(pd.read_csv(f"bert_gan_fake{i}.csv"))
    
fake = pd.concat(fakes)
fake['text'] = fake['text'].astype(str)
fake['label'] = 0
df = pd.concat([train, fake])[['text', 'label']].dropna()

In [ ]:
# split into test and val
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size=0.2, random_state=35)

train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)

print("Training Set Shape :", train.shape)
print("Validation Set Shape :", val.shape)

Input text pre-processing done; moving onto the model. (We'll still need to split the text into chunks though.)

In [ ]:
OUTPUT_DIR = 'tmp'

#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = True #@param {type:"boolean"}

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    pass

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

In [ ]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'

# label_list is the list of labels, i.e. True, False or 0, 1
label_list = [0, 1]

Split the text into chuncks (otherwise they're too big/BERT will grow exponentially).

In [ ]:
def get_split(text1):
    l_total = []
    l_parcial = []
    if len(text1.split())//150 >0:
        n = len(text1.split())//150
    else: 
        n = 1
    for w in range(n):
        if w == 0:
            l_parcial = text1.split()[:200]
            l_total.append(" ".join(l_parcial))
        else:
            l_parcial = text1.split()[w*150:w*150 + 200]
            l_total.append(" ".join(l_parcial))
    return l_total

In [ ]:
train['text_split'] = train[DATA_COLUMN].apply(get_split)
val['text_split'] = val[DATA_COLUMN].apply(get_split)

Go from array of chunks into dataframe of chunks.

In [ ]:
train_l = []
label_l = []
index_l =[]
for idx,row in train.iterrows():
    for l in row['text_split']:
        train_l.append(l)
        label_l.append(row['label'])
        index_l.append(idx)

train_df = pd.DataFrame({DATA_COLUMN:train_l, LABEL_COLUMN:label_l})
train_df.head()

In [ ]:
val_l = []
val_label_l = []
val_index_l = []
for idx,row in val.iterrows():
    for l in row['text_split']:
        val_l.append(l)
        val_label_l.append(row['label'])
        val_index_l.append(idx)

val_df = pd.DataFrame({DATA_COLUMN:val_l, LABEL_COLUMN:val_label_l})
val_df.head()

Text pre-processing truly finished; start moving data onto BERT.

In [ ]:
train_InputExamples = train_df.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = val_df.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [ ]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
    return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

In [ ]:
MAX_SEQ_LENGTH = 200

# Convert our train and validation features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(
    train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(
    val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

In [ ]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                                 num_labels):
    """Creates a classification model."""

    bert_module = hub.Module(
            BERT_MODEL_HUB,
            trainable=True)
    bert_inputs = dict(
            input_ids=input_ids,
            input_mask=input_mask,
            segment_ids=segment_ids)
    bert_outputs = bert_module(
            inputs=bert_inputs,
            signature="tokens",
            as_dict=True)

    # Use "pooled_output" for classification tasks on an entire sentence.
    # Use "sequence_outputs" for token-level output.
    output_layer = bert_outputs["pooled_output"]

    hidden_size = output_layer.shape[-1].value

    # Create our own layer to tune for politeness data.
    output_weights = tf.get_variable(
            "output_weights", [num_labels, hidden_size],
            initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable(
            "output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):

        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)


# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                                         num_warmup_steps):
    """Returns `model_fn` closure for TPUEstimator."""
    def model_fn(features, labels, mode, params):    # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
        
        # TRAIN and EVAL
        if not is_predicting:

            (loss, predicted_labels, log_probs) = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            train_op = bert.optimization.create_optimizer(
                    loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

            # Calculate evaluation metrics. 
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                f1_score = tf.contrib.metrics.f1_score(
                        label_ids,
                        predicted_labels)
                auc = tf.metrics.auc(
                        label_ids,
                        predicted_labels)
                recall = tf.metrics.recall(
                        label_ids,
                        predicted_labels)
                precision = tf.metrics.precision(
                        label_ids,
                        predicted_labels) 
                true_pos = tf.metrics.true_positives(
                        label_ids,
                        predicted_labels)
                true_neg = tf.metrics.true_negatives(
                        label_ids,
                        predicted_labels)     
                false_pos = tf.metrics.false_positives(
                        label_ids,
                        predicted_labels)    
                false_neg = tf.metrics.false_negatives(
                        label_ids,
                        predicted_labels)
                return {
                        "eval_accuracy": accuracy,
                        "f1_score": f1_score,
                        "auc": auc,
                        "precision": precision,
                        "recall": recall,
                        "true_positives": true_pos,
                        "true_negatives": true_neg,
                        "false_positives": false_pos,
                        "false_negatives": false_neg
                }

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode,
                    loss=loss,
                    train_op=train_op)
            else:
                    return tf.estimator.EstimatorSpec(mode=mode,
                        loss=loss,
                        eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            predictions = {
                    'probabilities': log_probs,
                    'labels': predicted_labels
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Return the actual model function in the closure
    return model_fn

In [ ]:
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [ ]:
#Initializing the model and the estimator
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

In [ ]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
#Training the model
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

In [ ]:
#Evaluating the model with Validation set
estimator.evaluate(input_fn=val_input_fn, steps=None)

Model trained, select best chunks.

In [ ]:
def getPrediction(in_sentence_pairs):
    labels = ["Fake", "Real"]
    input_examples = [run_classifier.InputExample(guid="", text_a = x[0], text_b = None , label = 0) 
                      for x in in_sentence_pairs] # here, "" is just a dummy label
    input_features = run_classifier.convert_examples_to_features(
        input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = run_classifier.input_fn_builder(
        features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
    predictions = estimator.predict(predict_input_fn)
    return pd.DataFrame([(sentence[0], sentence[1], np.exp(prediction['probabilities'][1]), 
                          labels[prediction['labels']]) 
                         for sentence, prediction in zip(in_sentence_pairs, predictions)], 
                        columns=['comment', 'reply', 'prob_real','label'])

In [ ]:
real_sent = test.loc[test['label']==1,['text']].values.tolist()
predictions_real = getPrediction(real_sent)
predictions_real.sort_values('prob_real')

In [ ]:
proposed_replies = pd.read_csv('TODO_proposed_replies.csv').dropna()
replies_to_test = proposed_replies[['text']].values.tolist()
predictions_proposed = getPrediction(replies_to_test)
predictions_proposed.sort_values('prob_real', ascending=False)

In [ ]:
best_predictions = (predictions_proposed.sort_values('prob_real', ascending=False)
                    .groupby('comment').first().reset_index())

In [ ]:
# return_table = (proposed_replies.drop(['proposed_reply','Unnamed: 0'], axis=1)
#                 .drop_duplicates().merge(best_predictions,on='comment'))

In [ ]:
best_predictions.loc[best_predictions['prob_real']>0.9].to_csv('realistic_replies.csv')

TODO: Go from chunks to document?? Maybe for each document average each chunk and only select docs above a given threshold?